In [1]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [2]:
from datetime import datetime, timedelta

import pandas as pd

current_date = pd.to_datetime(datetime.utcnow()).floor('h')
print(f"{current_date}")

2025-04-24 23:00:00


In [3]:
type(current_date)

pandas._libs.tslibs.timestamps.Timestamp

In [4]:
#from datetime import datetime, timedelta, timezone
#current_date = pd.to_datetime(datetime.now(timezone.utc)).floor("h")

In [5]:
#current_date.to_datetime64()

In [6]:
fetch_data_to = current_date
fetch_data_from = current_date - timedelta(days=29)

In [7]:
fetch_data_to

Timestamp('2025-04-24 23:00:00')

In [8]:
fetch_data_from

Timestamp('2025-03-26 23:00:00')

In [9]:
from datetime import datetime, timedelta
from typing import Union
import pandas as pd
from src.data_utils import load_and_process_taxi_data

def fetch_batch_raw_data(from_date: Union[datetime, str], to_date: Union[datetime, str]) -> pd.DataFrame:
    """
    Simulate production data by sampling historical data from 52 weeks ago (i.e., 1 year).

    Args:
        from_date (datetime or str): The start date for the data batch.
        to_date (datetime or str): The end date for the data batch.

    Returns:
        pd.DataFrame: A DataFrame containing the simulated production data.
    """
    # Convert string inputs to datetime if necessary
    if isinstance(from_date, str):
        from_date = datetime.fromisoformat(from_date)
    if isinstance(to_date, str):
        to_date = datetime.fromisoformat(to_date)

    # Validate input dates
    if not isinstance(from_date, datetime) or not isinstance(to_date, datetime):
        raise ValueError("Both 'from_date' and 'to_date' must be datetime objects or valid ISO format strings.")
    if from_date >= to_date:
        raise ValueError("'from_date' must be earlier than 'to_date'.")

    # Shift dates back by 52 weeks (1 year)
    historical_from_date = from_date - timedelta(weeks=52)
    historical_to_date = to_date - timedelta(weeks=52)

    # Load and filter data for the historical period
    rides_from = load_and_process_taxi_data(year=historical_from_date.year, months=[historical_from_date.month])
    rides_from = rides_from[rides_from.pickup_datetime >= historical_from_date]

    if historical_to_date.month != historical_from_date.month:
        rides_to = load_and_process_taxi_data(year=historical_to_date.year, months=[historical_to_date.month])
        rides_to = rides_to[rides_to.pickup_datetime < historical_to_date]
        # Combine the filtered data
        rides = pd.concat([rides_from, rides_to], ignore_index=True)
    else:
        rides = rides_from
    # Shift the data forward by 52 weeks to simulate recent data
    rides['pickup_datetime'] += timedelta(weeks=52)

    # Sort the data for consistency
    rides.sort_values(by=['pickup_location_id', 'pickup_datetime'], inplace=True)

    return rides

In [10]:
rides = fetch_batch_raw_data(fetch_data_from, fetch_data_to)

File already exists for 2024-03.
Loading data for 2024-03...
Total records: 3,582,628
Valid records: 3,518,066
Records dropped: 64,562 (1.80%)
Successfully processed data for 2024-03.
Combining all monthly data...
Data loading and processing complete!
Successfully downloaded data for 2024-04.
Loading data for 2024-04...
Total records: 3,514,289
Valid records: 3,450,929
Records dropped: 63,360 (1.80%)
Successfully processed data for 2024-04.
Combining all monthly data...
Data loading and processing complete!


In [11]:
rides

,pickup_datetime,pickup_location_id
543495,2025-03-31 23:48:03,2
2755678,2025-04-22 16:43:51,2
7163,2025-03-27 04:01:43,3
7067,2025-03-27 04:33:54,3
384432,2025-03-27 06:30:41,3
...,...,...
3000668,2025-04-24 22:54:33,263
3002192,2025-04-24 22:57:20,263
3005247,2025-04-24 22:58:11,263
3005347,2025-04-24 22:58:17,263


In [12]:
from src.data_utils import transform_raw_data_into_ts_data
ts_data = transform_raw_data_into_ts_data(rides)

In [13]:
ts_data

,pickup_hour,pickup_location_id,rides
0,2025-03-26 23:00:00,2,0
1,2025-03-27 00:00:00,2,0
2,2025-03-27 01:00:00,2,0
3,2025-03-27 02:00:00,2,0
4,2025-03-27 03:00:00,2,0
...,...,...,...
177475,2025-04-24 18:00:00,263,149
177476,2025-04-24 19:00:00,263,122
177477,2025-04-24 20:00:00,263,97
177478,2025-04-24 21:00:00,263,121


In [14]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 177480 entries, 0 to 177479
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   pickup_hour         177480 non-null  datetime64[ns]
 1   pickup_location_id  177480 non-null  int16         
 2   rides               177480 non-null  int16         
dtypes: datetime64[ns](1), int16(2)
memory usage: 2.0 MB


In [15]:
import hopsworks

# connect to the project
project = hopsworks.login(
    project=config.HOPSWORKS_PROJECT_NAME,
    api_key_value=config.HOPSWORKS_API_KEY
)

# connect to the feature store
feature_store = project.get_feature_store()

# connect to the feature group
feature_group = feature_store.get_feature_group(
    name=config.FEATURE_GROUP_NAME,
    version=config.FEATURE_GROUP_VERSION,
)



2025-04-24 19:21:54,523 INFO: Initializing external client
2025-04-24 19:21:54,524 INFO: Base URL: https://c.app.hopsworks.ai:443




To ensure compatibility please install the latest bug fix release matching the minor version of your backend (4.2) by running 'pip install hopsworks==4.2.*'


2025-04-24 19:21:55,507 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214677


In [16]:
feature_group.insert(ts_data, write_options={"wait_for_job": False})

Uploading Dataframe: 100.00% |██████████| Rows 177480/177480 | Elapsed Time: 00:07 | Remaining Time: 00:00
Use fg.materialization_job.run(args=-op offline_fg_materialization -path hdfs:///Projects/new_york_taxi/Resources/jobs/time_series_hourly_feature_group_1_offline_fg_materialization/config_1741010238064) to trigger the materialization job again.


(Job('time_series_hourly_feature_group_1_offline_fg_materialization', 'SPARK'),
 None)